In [1]:
import pandas as pd
import requests
import io
from ftfy import fix_encoding
import os
import sys
import datetime

#projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
import list_functions as lf

sourceDataDir = projectDir + "source-data/QLD/"
statusDir = projectDir + "source-data/status-codes/"
processedDataDir = projectDir + "current-lists/"
state = 'QLD'
monthStr = datetime.datetime.now().strftime('%Y%m')
codesfile = statusDir + state + "-codes.csv"
codesurl =  "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/species-status-codes.csv"
sensitivelisturl = "https://apps.des.qld.gov.au/data-sets/wildlife/wildnet/qld-confidential-species.csv"
prodListDruid = "dr493"
testListDruid = "dr18404"

## Download the raw files from data.qld.gov.au
... save locally

In [2]:
# Status codes
response = requests.get(codesurl)
rtext = fix_encoding(response.text)
speciescodes = pd.read_csv(io.StringIO(rtext))
speciescodes.to_csv(sourceDataDir + "species-status-codes.csv", index=False)

# Confidential/Sensitive List
response = requests.get(sensitivelisturl)
rtext = fix_encoding(response.text)
sensitivelist = pd.read_csv(io.StringIO(rtext))
sensitivelist.to_csv(sourceDataDir + "qld-confidential-species.csv", index=False)

... read local file

In [3]:
speciescodes = pd.read_csv(sourceDataDir + "species-status-codes.csv")
sensitivelist = pd.read_csv(sourceDataDir + "qld-confidential-species.csv")

## Standardise Status Codes
Some minimal changes to some Qld Nature Conservation Act codes so that they are consistent with other states

In [4]:
ncastatuscodes = speciescodes[speciescodes['Field'] == "NCA_status"][['Code', 'Code_description']]
ncastatuscodes['Code_description'] = ncastatuscodes['Code_description'].str.replace(" wildlife", "")
ncastatuscodes.loc[
    ncastatuscodes['Code_description'] == "Critically endangered", 'Code_description'] = "Critically Endangered"
ncastatuscodes.loc[ncastatuscodes['Code_description'] == "Near threatened", 'Code_description'] = "Near Threatened"
ncastatuscodes

,Code,Code_description
20,C,Least concern
21,CR,Critically Endangered
22,E,Endangered
23,EX,Extinct
24,I,International
25,NT,Near Threatened
26,P,Prohibited
27,PE,Extinct in the wild
28,SL,Special least concern
29,V,Vulnerable


## List processing
* Expand the nca status, endemicity and epbc status codes
* Rename fields to DwC terms
* Replace kingdom and class values with scientific terms

In [5]:
# nca status
sensitivelist = pd.merge(sensitivelist,ncastatuscodes,left_on=['NCA status'],right_on=['Code'],how="left")
sensitivelist = sensitivelist.rename(columns={'NCA status':'sourceStatus'})
sensitivelist = sensitivelist[['Scientific name', 'Common name', 'sourceStatus']]
sensitivelist = sensitivelist.rename(columns= {'Scientific name':'scientificName',
                                               'Common name': 'vernacularName',
                                                'sourceStatus': 'category' })
sensitivelist['generalisation'] = "2 km"
# map sourceStatus to category
codeMap = {'C': 'LC', 'CR': 'CR', 'E': 'EN',
           'NT': 'NT','PE': 'EW', 'SL': 'SL',
           'V': 'VU'}
sensitivelist['category'] = sensitivelist['category'].replace(codeMap)
sensitivelist['category'] = sensitivelist['category'].fillna('UK')
sensitivelist.to_csv(processedDataDir + 'sensitive-lists/QLD-' + prodListDruid + '-sensitive.csv',encoding="UTF-8",index=False)
sensitivelist

,scientificName,vernacularName,category,generalisation
0,Rhinolophus philippinensis,greater large-eared horseshoe bat,EN,2 km
1,Chloebia gouldiae,Gouldian finch,EN,2 km
2,Erythrura trichroa,blue-faced parrot-finch,NT,2 km
3,Neochmia phaeton evangelinae,crimson finch (white-bellied subspecies),LC,2 km
4,Poephila cincta cincta,black-throated finch (white-rumped subspecies),EN,2 km
...,...,...,...,...
955,Pneumatopteris pennigera,lime fern,EN,2 km
956,Reholttumia costata,NaN,NT,2 km
957,Thelypteris confluens,NaN,VU,2 km
958,Macadamia jansenii,NaN,CR,2 km


### Change Logs
Upload the file to the test environment before running the below cell to compare it to the list in production.
- check record counts old vs new and verify count in change log
- send to domain experts for verification

In [6]:
ltype = "S"
changeDir = "Monitoring/Change-logs/"
# Qld Sensitive
print('QLD sensitive')
filename = "QLD-sensitive.csv"
changelist = lf.get_changelist(testListDruid, prodListDruid, ltype)
changelist.to_csv(projectDir + changeDir + monthStr + "-" + filename, encoding="UTF-8", index=False)
changelist

QLD sensitive
get_changelist: Test -  dr18404 Prod -  dr493
download_ala_list:  https://lists.ala.org.au/ws/speciesListItems/dr493?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')
download_ala_list:  https://lists-test.ala.org.au/ws/speciesListItems/dr18404?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')


,name,scientificName_old,scientificName_new,commonName_old,commonName_new,listUpdate
28,Calorodius thorntonensis,NaN,None,NaN,None,added
105,Relicina rahengensis,NaN,Relicina rahengensis,NaN,None,added
278,Cooktownia,NaN,None,NaN,None,added
301,Corybas,NaN,None,NaN,None,added
324,Cymbidium,NaN,None,NaN,None,added
419,Dipodium,NaN,None,NaN,None,added
535,Liparis,NaN,None,NaN,None,added
540,Luisia,NaN,None,NaN,None,added
548,Microtis,NaN,None,NaN,None,added
576,Orthoceras,NaN,None,NaN,None,added


## Back up the list in production to `historical-lists`
If all ok, run the following to keep a copy of the list in prod before overwriting it

In [8]:
filename = state + "-" + prodListDruid + "-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + prodListDruid + "?max=10000&includeKVP=true"
prodList = lf.download_ala_specieslist(prodListUrl)
prodList = lf.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)

download_ala_list:  https://lists.ala.org.au/ws/speciesListItems/dr493?max=10000&includeKVP=true
Index(['id', 'name', 'commonName', 'scientificName', 'lsid', 'dataResourceUid',
       'kvpValues'],
      dtype='object')
